# Task 5: Group Analysis

After individual analysis, we will be performing analysing the US baby name dataframe as a team here. 
Jamie and Rita analyze the states level csv file and Leo analyze the National level csv file.
After discussion, we will be answering the three research questions as following:

### Research Question 1

What is the top 5 names from 1910 to 2015 in the US? How similar is it with the top 5 names in four US regions (North_East, South, West, Middle_West)?

### Research Question 2

Compare the most popular name at the state and national level in a decade: What is the most popular name in each year from 2000-2014?


### Research Question 3
What is the most popular national female name from 1910-2015? How about in each of the four regions? When does this name become popular (trend)? 
___

## Importing necessary modules and seperate data into 4 regions

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scripts import project_functions as pf

In [4]:
# Load 4 parts of raw data on State Names
state_df = pf.load_and_process_many("../../data/raw/state")

# Note that the project_fuctions module includes list of abbreviations for states separated in regions
# import the regional datasets
def get_state_df(state_list, source=state_df):
    return source.loc[source["State"].isin(state_list)].reset_index(drop=True)


n_df = get_state_df(pf.NORTH_EAST) # North East data set
s_df = get_state_df(pf.SOUTH)      # South data set
mw_df = get_state_df(pf.MID_WEST)  # Mid West data set
w_df = get_state_df(pf.WEST)    # West data set

In [13]:
# Sample data frame
w_df

,Name,Year,Gender,State,Count
0,Mary,1910,F,AK,14
1,Annie,1910,F,AK,12
2,Anna,1910,F,AK,10
3,Margaret,1910,F,AK,8
4,Helen,1910,F,AK,7
...,...,...,...,...,...
1156146,Seth,2014,M,WY,5
1156147,Spencer,2014,M,WY,5
1156148,Tyce,2014,M,WY,5
1156149,Victor,2014,M,WY,5


## Method to get all time top 5 in each region

In [42]:
# Define processing function
def get_top5_all_time(data=None):
    if data is None:
        return data
    return (data.groupby(by="Name").sum()
              .drop(columns=["Year"]) # We do not analyze with time
              .sort_values(by="Count", ascending=False)
              .reset_index()
              .head()
           )
# For the middle_west
top5_mw = get_top5_all_time(mw_df)
# For the west
top5_w = get_top5_all_time(w_df)
# For the North
top5_n = get_top5_all_time(n_df)
# For the south
top5_s = get_top5_all_time(s_df)

In [43]:
# Sample data frame
top5_w 

,Name,Count
0,Michael,748282
1,Robert,652150
2,David,647996
3,John,589500
4,James,545148


## Method to get all time top 5 based on gender

In [40]:
# Function for filter data based on gender
def get_top5_gender(data, region, gender):
    return (data.loc[data["Gender"] == gender]
                    .groupby(by="Name").sum()
                    .sort_values(by="Count", ascending=False)
                    .head()
                    .drop(columns="Year") # We do not care about year
                    .assign(Region=region, Gender=gender)
                    .reset_index()
            )

# In the North East
top5_male_n, top5_female_n = get_top5_gender(n_df, "NE", "M"), get_top5_gender(n_df, "NE", "F")

# In the South
top5_male_s, top5_female_s = get_top5_gender(s_df, "S", "M"), get_top5_gender(s_df, "S", "F")

# In the West
top5_male_w, top5_female_w = get_top5_gender(w_df, "W", "M"), get_top5_gender(w_df, "W", "F")

# In the Mid West
top5_male_mw, top5_female_mw = get_top5_gender(mw_df, "MW", "M"), get_top5_gender(mw_df, "MW", "F")                             

In [41]:
# Sample data frame
top5_male_w

,Name,Count,Region,Gender
0,Michael,744895,W,M
1,Robert,650605,W,M
2,David,646280,W,M
3,John,588235,W,M
4,James,543933,W,M


In [32]:
top5_female_w

,Name,Count,Region,Gender
0,Mary,339339,W,F
1,Jennifer,301413,W,F
2,Jessica,234652,W,F
3,Elizabeth,228588,W,F
4,Patricia,208413,W,F


## Method to get most popular female name in 1910 and its proportion from 1910 to 2015

In [38]:
def get_top_1910_fname(data, region=None):
    if data is None:
        return data
    top_fname = (data.loc[data["Year"] == 1910]
                   .groupby(by="Name").sum()
                   .drop(columns="Year")
                   .sort_values(by="Count", ascending=False)
                   .index[0]
                )
    # Pivot table of Years vs Name 
    fname_year = (data.loc[data["Name"] == top_fname]
                .pivot_table(index="Year",columns="Name",values="Count", aggfunc=np.sum)
            )
    # Find the total count in each year
    y = data.groupby(by="Year").agg(np.sum)

    # Divide each row (year) by the total of that year
    for year in range(1910, 2015):
        fname_year.loc[year,:] = fname_year.loc[year,:]/y.loc[year,"Count"]

    return fname_year

# Top female name of North East in 1910 and its trend
f_name_n = get_top_1910_fname(data=n_df, region="NE")

# Top female name of North East in 1910 and its trend
f_name_s = get_top_1910_fname(data=s_df, region="S")

# Top female name of North East in 1910 and its trend
f_name_w = get_top_1910_fname(data=w_df, region="W")

# Top female name of North East in 1910 and its trend
f_name_mw = get_top_1910_fname(data=mw_df, region="MW")


In [39]:
# Sample data frame
f_name_w 

Name,Mary
Year,
1910,0.042355
1911,0.042245
1912,0.034384
1913,0.030812
1914,0.030345
...,...
2010,0.000531
2011,0.000519
2012,0.000493


## Method to get most popular name in a decade

In [80]:
def get_top_in_interval(data, begin, stop):
    top = list()
    # Extract only the year in interest
    for year in range(begin, stop+1):
        top_in_year =   (data.loc[data["Year"] == year]
                         .groupby(by="Name").sum()
                         .sort_values(by="Count", ascending=False)
                         .assign(Year=year)
                         .reset_index()
                         .iloc[0]
                        )
        top.append(top_in_year)

    top_in_interval = pd.concat(top, axis=1, ignore_index=True).T
    return top_in_interval
        
get_top_in_interval(s_df, begin=2000, stop=2014)

,Name,Year,Count
0,Jacob,2000,12405
1,Jacob,2001,11839
2,Jacob,2002,11228
3,Jacob,2003,11063
4,Jacob,2004,10611
5,Jacob,2005,9699
6,Jacob,2006,9351
7,Jacob,2007,9222
8,Jacob,2008,8424
9,Jayden,2009,7982


## Method from graph (references)

- You can find in Rita or Jamie's notebook!